In [12]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import PIL
from keras.models import load_model
import cv2
import numpy as np 
import os

In [2]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [3]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
training_path = r'C:\Users\sonuc\Desktop\Data_Science\Malaria_detection\Malaria Cells\training_set'
testing_path = r'C:\Users\sonuc\Desktop\Data_Science\Malaria_detection\Malaria Cells\testing_set'

In [21]:
batch_size = 10 

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data 
train_generator = train_datagen.flow_from_directory(
        training_path,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size)
        #class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=(150, 150),
        batch_size=batch_size)#,
        #class_mode='binary')

Found 27558 images belonging to 2 classes.
Found 15832 images belonging to 2 classes.


In [23]:
model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

model.save('malaria_detect.h5')  # always save your weights after training or during training

Epoch 1/50
200/200 [==============================] - 11s 54ms/step - loss: 0.3659 - accuracy: 0.8755 - val_loss: 0.6373 - val_accuracy: 0.6237
Epoch 2/50
200/200 [==============================] - 11s 54ms/step - loss: 0.3525 - accuracy: 0.8770 - val_loss: 0.1608 - val_accuracy: 0.9362
Epoch 3/50
200/200 [==============================] - 11s 54ms/step - loss: 0.3154 - accuracy: 0.9005 - val_loss: 0.1966 - val_accuracy: 0.9300
Epoch 4/50
200/200 [==============================] - 11s 54ms/step - loss: 0.3516 - accuracy: 0.8680 - val_loss: 0.1593 - val_accuracy: 0.9513
Epoch 5/50
200/200 [==============================] - 11s 54ms/step - loss: 0.2631 - accuracy: 0.9205 - val_loss: 0.1582 - val_accuracy: 0.9488
Epoch 6/50
200/200 [==============================] - 11s 54ms/step - loss: 0.2649 - accuracy: 0.9240 - val_loss: 0.2193 - val_accuracy: 0.9075
Epoch 7/50
200/200 [==============================] - 11s 54ms/step - loss: 0.2414 - accuracy: 0.9210 - val_loss: 0.1771 - val_accuracy:

In [ ]:
model = load_model('malaria_detect.h5') 

In [34]:
image_path = r'C:\Users\sonuc\Desktop\Data_Science\Malaria_detection\Malaria Cells\single_prediction\2p.png'
image_path

'C:\\Users\\sonuc\\Desktop\\Data_Science\\Malaria_detection\\Malaria Cells\\single_prediction\\2p.png'

In [35]:
user_img = image.load_img(image_path,target_size = (150,150))
user_img = image.img_to_array(user_img)
user_img = user_img.reshape(1, 150, 150, 3)


x = (model.predict(user_img) > 0.5).astype("int32")
#print(x)
if(x[0][0]== 0):      
    label = 'Uninfected'
    print(label)
else:
    label = 'Parasitised'
    print(label)

Parasitised
